In [1]:
import os

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import itertools
from pylab import rcParams
import matplotlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt
import sklearn
import sklearn.datasets
import sklearn.linear_model
from numpy import genfromtxt
import tensorflow as tf
from keras.layers import Activation, Dense

from tensorflow.contrib import learn
from sklearn import cross_validation
from sklearn import preprocessing
from sklearn import metrics


%matplotlib inline

np.random.seed(1) # set a seed so that the results are consistent

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [23]:
path = "./data/"

In [2]:
train = pd.read_csv('data/Train_data/dataset_rev4_train.csv')
#buffer datetime
buffer_datetime_train = train.datetime
#remove object
train = train.select_dtypes(exclude=['object'])
#replace misssing value
train.fillna(0,inplace=True)

In [3]:
test = pd.read_csv('data/test_data/dataset_rev4_test.csv')
#buffer datetime
buffer_datetime_test = test.datetime
#remove object
test = test.select_dtypes(exclude=['object'])
#replace misssing value
test.fillna(0,inplace=True)

In [4]:
print('dimension of train:', train.shape)
print('dimension of test:', test.shape)

dimension of train: (131586, 6)
dimension of test: (16147, 5)


In [5]:
print("features:",list(train.columns))

features: ['I', 'T', 'UV', 'WS', 'RH', 'P']


In [6]:
from sklearn.ensemble import IsolationForest
clf = IsolationForest(max_samples = 100, random_state = 42)
clf.fit(train)
y_noano = clf.predict(train)
y_noano = pd.DataFrame(y_noano, columns = ['Top'])
y_noano[y_noano['Top'] == 1].index.values

train = train.iloc[y_noano[y_noano['Top'] == 1].index.values]
train.reset_index(drop = True, inplace = True)
print("Number of Outliers:", y_noano[y_noano['Top'] == -1].shape[0])
print("Number of rows without outliers:", train.shape[0])

Number of Outliers: 13159
Number of rows without outliers: 118427


In [7]:
import warnings
warnings.filterwarnings('ignore')

col_train = list(train.columns)
col_train_bis = list(train.columns)

col_train_bis.remove('P')

mat_train = np.matrix(train)
mat_test  = np.matrix(test)

mat_new = np.matrix(train.drop('P',axis = 1))
mat_y = np.array(train.P).reshape((118427,1))

prepro_y = MinMaxScaler()
prepro_y.fit(mat_y)

prepro = MinMaxScaler()
prepro.fit(mat_train)

prepro_test = MinMaxScaler()
prepro_test.fit(mat_new)

train = pd.DataFrame(prepro.transform(mat_train),columns = col_train)
test  = pd.DataFrame(prepro_test.transform(mat_test),columns = col_train_bis)

In [8]:
# List of features
COLUMNS = col_train #column train (x train)
FEATURES = col_train_bis  #column train-label (x test)
LABEL = "P"

# Columns
feature_cols = FEATURES #(x test)

# Training set and Prediction set with the features to predict
training_set = train[COLUMNS] #column train (x train)
prediction_set = train.P # column P

In [9]:
print(type(training_set))
print(type(prediction_set))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>


In [10]:
x_train, x_test, y_train, y_test = train_test_split(training_set[FEATURES] , prediction_set, test_size=0.33, random_state=42)

In [11]:
train_data = x_train.values
train_labels = y_train.values
test_data = x_test.values
test_labels = y_test.values

In [13]:
print(type(train_data))
print(type(train_labels))
print(type(test_data))
print(type(test_labels))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [14]:
from keras import models
from keras import layers

original_model = models.Sequential()
original_model.add(layers.Dense(16, activation='relu', input_shape=(5,)))
original_model.add(layers.Dense(16, activation='relu'))
original_model.add(layers.Dense(1, activation='relu'))

original_model.compile(optimizer='RMSprop',
                       loss='mse',
                       metrics=['mae'])

In [15]:
smaller_model = models.Sequential()
smaller_model.add(layers.Dense(4, activation='relu', input_shape=(5,)))
smaller_model.add(layers.Dense(4, activation='relu'))
smaller_model.add(layers.Dense(1, activation='relu'))

smaller_model.compile(optimizer='rmsprop',
                      loss='mse',
                      metrics=['mae'])

In [16]:
original_hist = original_model.fit(x_train, y_train,
                                   epochs=20,
                                   batch_size=512,
                                   validation_data=(x_test, y_test))

Train on 79346 samples, validate on 39081 samples
Epoch 1/20
79346/79346 [==============================] - 1s 7us/step - loss: 0.0105 - mean_absolute_error: 0.0669 - val_loss: 0.0044 - val_mean_absolute_error: 0.0414
Epoch 2/20
79346/79346 [==============================] - 0s 5us/step - loss: 0.0042 - mean_absolute_error: 0.0394 - val_loss: 0.0040 - val_mean_absolute_error: 0.0365
Epoch 3/20
79346/79346 [==============================] - 0s 5us/step - loss: 0.0041 - mean_absolute_error: 0.0388 - val_loss: 0.0048 - val_mean_absolute_error: 0.0487
Epoch 4/20
79346/79346 [==============================] - 0s 5us/step - loss: 0.0040 - mean_absolute_error: 0.0384 - val_loss: 0.0043 - val_mean_absolute_error: 0.0432
Epoch 5/20
79346/79346 [==============================] - 0s 5us/step - loss: 0.0040 - mean_absolute_error: 0.0382 - val_loss: 0.0038 - val_mean_absolute_error: 0.0363
Epoch 6/20
79346/79346 [==============================] - 0s 4us/step - loss: 0.0039 - mean_absolute_error: 0.

In [19]:
# Predict# Predic 
pred = original_model.predict(test_data)

In [20]:
# Measure RMSE error.  RMSE is common for regression.
score = np.sqrt(metrics.mean_squared_error(pred,test_labels))
print(f"Before save score (RMSE): {score}")

Before save score (RMSE): 0.059894907030809255


In [24]:
# save neural network structure to JSON (no weights)
model_json = original_model.to_json()
with open(os.path.join(path,"R16_1_model.json"), "w") as json_file:
    json_file.write(model_json)

In [26]:
# save neural network structure to YAML (no weights)
model_yaml = original_model.to_yaml()
with open(os.path.join(path,"R16_1_model.yaml"), "w") as yaml_file:
    yaml_file.write(model_yaml)

In [27]:
# save entire network to HDF5 (save everything, suggested)# save e 
original_model.save(os.path.join(path,"R16_1_model.h5"))

In [38]:
from keras.models import load_model
model2 = load_model(os.path.join(path,"R16_1_model.h5"))
pred = model2.predict(test_data)
# Measure RMSE error.  RMSE is common for regression.
score = np.sqrt(metrics.mean_squared_error(pred,test_labels))
print(f"After load score (RMSE): {score}")

After load score (RMSE): 0.059894907030809255


In [28]:
smaller_model_hist = smaller_model.fit(x_train, y_train,
                                       epochs=20,
                                       batch_size=512,
                                       validation_data=(x_test, y_test))

Train on 79346 samples, validate on 39081 samples
Epoch 1/20
79346/79346 [==============================] - 0s 6us/step - loss: 0.1230 - mean_absolute_error: 0.2673 - val_loss: 0.1218 - val_mean_absolute_error: 0.2660
Epoch 2/20
79346/79346 [==============================] - 0s 4us/step - loss: 0.1230 - mean_absolute_error: 0.2673 - val_loss: 0.1218 - val_mean_absolute_error: 0.2660
Epoch 3/20
79346/79346 [==============================] - 0s 5us/step - loss: 0.1230 - mean_absolute_error: 0.2673 - val_loss: 0.1218 - val_mean_absolute_error: 0.2660
Epoch 4/20
79346/79346 [==============================] - 0s 5us/step - loss: 0.1230 - mean_absolute_error: 0.2673 - val_loss: 0.1218 - val_mean_absolute_error: 0.2660
Epoch 5/20
79346/79346 [==============================] - 0s 5us/step - loss: 0.1230 - mean_absolute_error: 0.2673 - val_loss: 0.1218 - val_mean_absolute_error: 0.2660
Epoch 6/20
79346/79346 [==============================] - 0s 5us/step - loss: 0.1230 - mean_absolute_error: 0.

In [29]:
# Predict# Predic 
pred = smaller_model.predict(test_data)

In [30]:
# Measure RMSE error.  RMSE is common for regression.
score = np.sqrt(metrics.mean_squared_error(pred,test_labels))
print(f"Before save score (RMSE): {score}")

Before save score (RMSE): 0.34894193043374266


In [34]:
# save neural network structure to JSON (no weights)
model_json = smaller_model.to_json()
with open(os.path.join(path,"R16_2_model.json"), "w") as json_file:
    json_file.write(model_json)

In [35]:
# save neural network structure to YAML (no weights)
model_yaml = smaller_model.to_yaml()
with open(os.path.join(path,"R16_2_model.yaml"), "w") as yaml_file:
    yaml_file.write(model_yaml)

In [36]:
# save entire network to HDF5 (save everything, suggested)# save e 
smaller_model.save(os.path.join(path,"R16_2_model.h5"))

In [37]:
from keras.models import load_model
model2 = load_model(os.path.join(path,"R16_2_model.h5"))
pred = model2.predict(test_data)
# Measure RMSE error.  RMSE is common for regression.
score = np.sqrt(metrics.mean_squared_error(pred,test_labels))
print(f"After load score (RMSE): {score}")

After load score (RMSE): 0.34894193043374266


In [39]:
bigger_model = models.Sequential()
bigger_model.add(layers.Dense(512, activation='relu', input_shape=(5,)))
bigger_model.add(layers.Dense(512, activation='relu'))
bigger_model.add(layers.Dense(1, activation='relu'))

bigger_model.compile(optimizer='rmsprop',
                     loss='mse',
                     metrics=['mae'])

In [49]:
bigger_model_hist = bigger_model.fit(x_train, y_train,
                                     epochs=20,
                                     batch_size=512,
                                     validation_data=(x_test, y_test))

Train on 79346 samples, validate on 39081 samples
Epoch 1/20
79346/79346 [==============================] - 4s 47us/step - loss: 0.0035 - mean_absolute_error: 0.0331 - val_loss: 0.0033 - val_mean_absolute_error: 0.0313
Epoch 2/20
79346/79346 [==============================] - 3s 44us/step - loss: 0.0035 - mean_absolute_error: 0.0329 - val_loss: 0.0035 - val_mean_absolute_error: 0.0321
Epoch 3/20
79346/79346 [==============================] - 4s 47us/step - loss: 0.0035 - mean_absolute_error: 0.0330 - val_loss: 0.0035 - val_mean_absolute_error: 0.0326
Epoch 4/20
79346/79346 [==============================] - 4s 47us/step - loss: 0.0034 - mean_absolute_error: 0.0328 - val_loss: 0.0035 - val_mean_absolute_error: 0.0326
Epoch 5/20
79346/79346 [==============================] - 4s 46us/step - loss: 0.0034 - mean_absolute_error: 0.0327 - val_loss: 0.0033 - val_mean_absolute_error: 0.0318
Epoch 6/20
79346/79346 [==============================] - 4s 47us/step - loss: 0.0034 - mean_absolute_err

In [41]:
# Predict# Predic 
pred = bigger_model.predict(test_data)

In [42]:
# Measure RMSE error.  RMSE is common for regression.
score = np.sqrt(metrics.mean_squared_error(pred,test_labels))
print(f"Before save score (RMSE): {score}")

Before save score (RMSE): 0.06014507683603035


In [43]:
# save neural network structure to JSON (no weights)
model_json = bigger_model.to_json()
with open(os.path.join(path,"R16_3_model.json"), "w") as json_file:
    json_file.write(model_json)

In [44]:
# save neural network structure to YAML (no weights)
model_yaml = bigger_model.to_yaml()
with open(os.path.join(path,"R16_3_model.yaml"), "w") as yaml_file:
    yaml_file.write(model_yaml)

In [45]:
# save entire network to HDF5 (save everything, suggested)# save e 
bigger_model.save(os.path.join(path,"R16_3_model.h5"))

In [46]:
from keras.models import load_model
model2 = load_model(os.path.join(path,"R16_3_model.h5"))
pred = model2.predict(test_data)
# Measure RMSE error.  RMSE is common for regression.
score = np.sqrt(metrics.mean_squared_error(pred,test_labels))
print(f"After load score (RMSE): {score}")

After load score (RMSE): 0.06014507683603035


In [47]:
from keras import regularizers

l2_model = models.Sequential()
l2_model.add(layers.Dense(16, kernel_regularizer=regularizers.l2(0.001),
                          activation='relu', input_shape=(5,)))
l2_model.add(layers.Dense(16, kernel_regularizer=regularizers.l2(0.001),
                          activation='relu'))
l2_model.add(layers.Dense(1, activation='relu'))

In [50]:
l2_model.compile(optimizer='rmsprop',
                 loss='mse',
                 metrics=['mae'])

In [51]:
l2_model_hist = l2_model.fit(x_train, y_train,
                             epochs=20,
                             batch_size=512,
                             validation_data=(x_test, y_test))

Train on 79346 samples, validate on 39081 samples
Epoch 1/20
79346/79346 [==============================] - 1s 7us/step - loss: 0.0279 - mean_absolute_error: 0.0620 - val_loss: 0.0184 - val_mean_absolute_error: 0.0419
Epoch 2/20
79346/79346 [==============================] - 0s 3us/step - loss: 0.0150 - mean_absolute_error: 0.0397 - val_loss: 0.0128 - val_mean_absolute_error: 0.0414
Epoch 3/20
79346/79346 [==============================] - 0s 4us/step - loss: 0.0112 - mean_absolute_error: 0.0390 - val_loss: 0.0102 - val_mean_absolute_error: 0.0375
Epoch 4/20
79346/79346 [==============================] - 0s 4us/step - loss: 0.0096 - mean_absolute_error: 0.0386 - val_loss: 0.0090 - val_mean_absolute_error: 0.0397
Epoch 5/20
79346/79346 [==============================] - 0s 3us/step - loss: 0.0086 - mean_absolute_error: 0.0383 - val_loss: 0.0087 - val_mean_absolute_error: 0.0428
Epoch 6/20
79346/79346 [==============================] - 0s 3us/step - loss: 0.0078 - mean_absolute_error: 0.

In [52]:
# Predict# Predic 
pred = l2_model.predict(test_data)

In [53]:
# Measure RMSE error.  RMSE is common for regression.
score = np.sqrt(metrics.mean_squared_error(pred,test_labels))
print(f"Before save score (RMSE): {score}")

Before save score (RMSE): 0.06179037023349529


In [54]:
# save neural network structure to JSON (no weights)
model_json = l2_model.to_json()
with open(os.path.join(path,"R16_4_model.json"), "w") as json_file:
    json_file.write(model_json)

In [55]:
# save neural network structure to YAML (no weights)
model_yaml = l2_model.to_yaml()
with open(os.path.join(path,"R16_4_model.yaml"), "w") as yaml_file:
    yaml_file.write(model_yaml)

In [56]:
# save entire network to HDF5 (save everything, suggested)# save e 
l2_model.save(os.path.join(path,"R16_4_model.h5"))

In [57]:
from keras.models import load_model
model2 = load_model(os.path.join(path,"R16_4_model.h5"))
pred = model2.predict(test_data)
# Measure RMSE error.  RMSE is common for regression.
score = np.sqrt(metrics.mean_squared_error(pred,test_labels))
print(f"After load score (RMSE): {score}")

After load score (RMSE): 0.06179037023349529


In [58]:
from keras import regularizers

# L1 regularization
regularizers.l1(0.001)

# L1 and L2 regularization at the same time
regularizers.l1_l2(l1=0.001, l2=0.001)

In [59]:
dpt_model = models.Sequential()
dpt_model.add(layers.Dense(16, activation='relu', input_shape=(5,)))
dpt_model.add(layers.Dropout(0.5))
dpt_model.add(layers.Dense(16, activation='relu'))
dpt_model.add(layers.Dropout(0.5))
dpt_model.add(layers.Dense(1, activation='relu'))

dpt_model.compile(optimizer='rmsprop',
                  loss='mse',
                  metrics=['mae'])

In [60]:
dpt_model_hist = dpt_model.fit(x_train, y_train,
                               epochs=20,
                               batch_size=512,
                               validation_data=(x_test, y_test))

Train on 79346 samples, validate on 39081 samples
Epoch 1/20
79346/79346 [==============================] - 1s 10us/step - loss: 0.0650 - mean_absolute_error: 0.1898 - val_loss: 0.0173 - val_mean_absolute_error: 0.1030
Epoch 2/20
79346/79346 [==============================] - 0s 6us/step - loss: 0.0228 - mean_absolute_error: 0.1128 - val_loss: 0.0093 - val_mean_absolute_error: 0.0677
Epoch 3/20
79346/79346 [==============================] - 0s 5us/step - loss: 0.0166 - mean_absolute_error: 0.0954 - val_loss: 0.0073 - val_mean_absolute_error: 0.0606
Epoch 4/20
79346/79346 [==============================] - 0s 6us/step - loss: 0.0138 - mean_absolute_error: 0.0860 - val_loss: 0.0069 - val_mean_absolute_error: 0.0588
Epoch 5/20
79346/79346 [==============================] - 0s 6us/step - loss: 0.0126 - mean_absolute_error: 0.0819 - val_loss: 0.0068 - val_mean_absolute_error: 0.0597
Epoch 6/20
79346/79346 [==============================] - 0s 6us/step - loss: 0.0120 - mean_absolute_error: 0

In [68]:
# Predict# Predic 
pred = dpt_model.predict(test_data)

In [69]:
# Measure RMSE error.  RMSE is common for regression.
score = np.sqrt(metrics.mean_squared_error(pred,test_labels))
print(f"Before save score (RMSE): {score}")

Before save score (RMSE): 0.07976824504799956


In [70]:
# save neural network structure to JSON (no weights)
model_json = dpt_model.to_json()
with open(os.path.join(path,"R14_5_model.json"), "w") as json_file:
    json_file.write(model_json)

In [71]:
# save neural network structure to YAML (no weights)
model_yaml = dpt_model.to_yaml()
with open(os.path.join(path,"R14_5_model.yaml"), "w") as yaml_file:
    yaml_file.write(model_yaml)

In [72]:
# save entire network to HDF5 (save everything, suggested)# save e 
dpt_model.save(os.path.join(path,"R14_5_model.h5"))

In [73]:
from keras.models import load_model
model2 = load_model(os.path.join(path,"R14_5_model.h5"))
pred = model2.predict(test_data)
# Measure RMSE error.  RMSE is common for regression.
score = np.sqrt(metrics.mean_squared_error(pred,test_labels))
print(f"After load score (RMSE): {score}")

After load score (RMSE): 0.07976824504799956
